# 고차원 텐서 완전 가이드

## LLM 프로젝트에서 자주 사용되는 텐서 연산 마스터하기

이 노트북에서는 PyTorch 텐서의 핵심 개념과 연산들을 단계별로 학습합니다:
- 차원 분해 및 Shape 이해
- Squeeze/Unsqueeze로 차원 조작
- **전치(Transpose)**: 차원 재배열
- **점곱(Dot Product)**: 벡터 및 행렬 연산
- Broadcasting 규칙
- LLM 실전 예시

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# 한글 폰트 설정
plt.rcParams['font.family'] = 'DejaVu Sans'
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

## 1️⃣ 차원 분해 (Shape Breakdown)

고차원 텐서의 각 차원이 **무엇을 의미하는지** 명확히 이해하기

In [ ]:
# 예: GPT의 어텐션 출력
batch_size = 2
seq_length = 1024
emb_dim = 768

attention_output = torch.randn(batch_size, seq_length, emb_dim)

print("=== 어텐션 출력 텐서 ===")
print(f"Shape: {attention_output.shape}")
print(f"차원 0 (배치): {attention_output.shape[0]}개 문서")
print(f"차원 1 (시퀀스): {attention_output.shape[1]}개 토큰")
print(f"차원 2 (임베딩): {attention_output.shape[2]}차원 벡터")
print(f"\n총 원소 수: {attention_output.numel()}")
print(f"메모리 크기: {attention_output.element_size() * attention_output.numel() / 1024 / 1024:.2f} MB")

## 2️⃣ 특정 차원에 집중 (Indexing)

한 번에 하나의 차원만 선택해서 생각하기

In [ ]:
# 작은 텐서로 실험
x = torch.randn(2, 5, 3)  # (배치=2, 토큰=5, 차원=3)
print("원본 텐서:")
print(f"Shape: {x.shape}")
print(f"내용:\n{x}")

print("\n=== 배치 0만 선택 ===")
batch_0 = x[0]  # (5, 3)
print(f"Shape: {batch_0.shape}")
print(f"내용:\n{batch_0}")

print("\n=== 배치 0의 첫 토큰 ===")
first_token = x[0, 0]  # (3,)
print(f"Shape: {first_token.shape}")
print(f"내용: {first_token}")

print("\n=== 모든 배치의 첫 토큰 ===")
first_tokens = x[:, 0, :]  # (2, 3)
print(f"Shape: {first_tokens.shape}")
print(f"내용:\n{first_tokens}")

## 3️⃣ Squeeze & Unsqueeze (차원 제거/추가)

불필요한 차원을 제거하거나 새 차원을 추가하기

In [ ]:
# Squeeze: 크기가 1인 차원 제거
x_1 = torch.randn(1, 10, 768)  # (배치=1, 토큰=10, 임베딩=768)
print("=== Squeeze (차원 제거) ===")
print(f"원본 shape: {x_1.shape}")
x_1_squeezed = x_1.squeeze()  # 모든 크기 1인 차원 제거
print(f"squeeze() 후: {x_1_squeezed.shape}")

# 특정 차원만 제거
x_1_squeezed_0 = x_1.squeeze(0)  # 차원 0만 제거
print(f"squeeze(0) 후: {x_1_squeezed_0.shape}")

# Unsqueeze: 새 차원 추가
print("\n=== Unsqueeze (차원 추가) ===")
x_2 = torch.randn(10, 768)  # (토큰=10, 임베딩=768)
print(f"원본 shape: {x_2.shape}")
x_2_unsqueezed_0 = x_2.unsqueeze(0)  # 맨 앞에 차원 추가
print(f"unsqueeze(0) 후: {x_2_unsqueezed_0.shape}")  # (1, 10, 768)
x_2_unsqueezed_1 = x_2.unsqueeze(1)  # 중간에 차원 추가
print(f"unsqueeze(1) 후: {x_2_unsqueezed_1.shape}")  # (10, 1, 768)

## 4️⃣ 전치 (Transpose/Permute) 🔄

텐서의 차원 순서를 바꾸기. 어텐션, 행렬 곱셈 등에서 자주 사용됩니다.

In [ ]:
# 2D 행렬의 전치
print("=== 2D 행렬 전치 ===")
matrix = torch.arange(6).reshape(2, 3)
print(f"원본 (2, 3):\n{matrix}")
transposed = matrix.T  # 또는 matrix.transpose(0, 1)
print(f"\n전치 후 (3, 2):\n{transposed}")

In [ ]:
# 고차원 텐서의 전치 (Permute)
print("\n=== 고차원 텐서 전치 (Permute) ===")
tensor = torch.randn(2, 3, 4, 5)  # (배치, 토큰, 헤드, 차원)
print(f"원본 shape: {tensor.shape}")

# 차원 0과 2를 교환
permuted = tensor.permute(0, 2, 1, 3)  # (배치, 헤드, 토큰, 차원)
print(f"permute(0, 2, 1, 3) 후: {permuted.shape}")

# 모든 차원을 역순으로
reversed_dims = tensor.permute(3, 2, 1, 0)  # (차원, 헤드, 토큰, 배치)
print(f"permute(3, 2, 1, 0) 후: {reversed_dims.shape}")

In [ ]:
# 실전: 멀티헤드 어텐션에서의 전치
print("\n=== LLM 실전 예시: 멀티헤드 어텐션 ===")
batch_size, seq_len, emb_dim, n_heads = 2, 4, 8, 2
head_dim = emb_dim // n_heads

# Q, K, V를 head 형태로 변환: (batch, seq, emb_dim) → (batch, seq, n_heads, head_dim)
Q = torch.randn(batch_size, seq_len, emb_dim)
print(f"Q 원본: {Q.shape}")

Q_heads = Q.reshape(batch_size, seq_len, n_heads, head_dim)
print(f"Q reshape 후: {Q_heads.shape}")

# 헤드 계산을 위해 차원 재배열: (batch, seq, n_heads, head_dim) → (batch, n_heads, seq, head_dim)
Q_heads = Q_heads.permute(0, 2, 1, 3)
print(f"Q permute(0, 2, 1, 3) 후: {Q_heads.shape}")
print("\n이제 각 헤드별로 병렬로 어텐션 연산 가능!")

## 5️⃣ 점곱 (Dot Product) 🔹

두 벡터 또는 행렬의 내적. 어텐션 메커니즘의 핵심입니다.

In [ ]:
# 기본: 1D 벡터의 점곱
print("=== 1D 벡터 점곱 ===")
v1 = torch.tensor([1.0, 2.0, 3.0])
v2 = torch.tensor([4.0, 5.0, 6.0])

# 방법 1: dot()
dot_result = torch.dot(v1, v2)
print(f"v1: {v1}")
print(f"v2: {v2}")
print(f"torch.dot(v1, v2) = {dot_result}")
print(f"수동 계산: 1×4 + 2×5 + 3×6 = {1*4 + 2*5 + 3*6}")

# 방법 2: @ 연산자
dot_result_2 = v1 @ v2
print(f"v1 @ v2 = {dot_result_2}")

In [ ]:
# 2D 행렬 곱셈 (Matrix Multiplication)
print("\n=== 2D 행렬 곱셈 ===")
A = torch.tensor([[1.0, 2.0],
                   [3.0, 4.0],
                   [5.0, 6.0]])  # (3, 2)
B = torch.tensor([[7.0, 8.0, 9.0],
                   [10.0, 11.0, 12.0]])  # (2, 3)

print(f"A shape: {A.shape}")
print(f"B shape: {B.shape}")
print(f"A @ B shape: {(A @ B).shape}")

C = A @ B  # (3, 2) @ (2, 3) = (3, 3)
print(f"\nA @ B =\n{C}")

In [ ]:
# 고차원 배치 행렬 곱셈
print("\n=== 배치 행렬 곱셈 (Batch Matrix Multiplication) ===")
X = torch.randn(4, 3, 2)  # (배치=4, 시퀀스=3, 특성=2)
W = torch.randn(2, 5)      # (입력=2, 출력=5)

print(f"X shape: {X.shape}")
print(f"W shape: {W.shape}")

Y = X @ W  # (4, 3, 2) @ (2, 5) = (4, 3, 5)
print(f"X @ W shape: {Y.shape}")
print(f"\n배치와 시퀀스는 유지되고, 마지막 차원만 변환됨!")

In [ ]:
# 실전: 어텐션 스코어 계산
print("\n=== LLM 실전 예시: 어텐션 스코어 ===")
batch_size, seq_len, head_dim = 2, 4, 3

# Query와 Key 텐서 (단순화)
Q = torch.randn(batch_size, seq_len, head_dim)  # (2, 4, 3)
K = torch.randn(batch_size, seq_len, head_dim)  # (2, 4, 3)

print(f"Q shape: {Q.shape}")
print(f"K shape: {K.shape}")

# Key를 전치
K_T = K.transpose(-2, -1)  # (2, 3, 4) - 마지막 두 차원을 바꿈
print(f"K.transpose(-2, -1) shape: {K_T.shape}")

# 어텐션 스코어 = Q @ K^T
attention_scores = Q @ K_T  # (2, 4, 3) @ (2, 3, 4) = (2, 4, 4)
print(f"Attention scores shape: {attention_scores.shape}")
print(f"\n각 쿼리가 모든 키와의 유사도를 계산했습니다!")
print(f"shape [2, 4, 4] = [배치, 쿼리_위치, 키_위치]")

## 6️⃣ Broadcasting 규칙

크기가 다른 텐서들을 자동으로 맞춰서 연산하기

In [ ]:
# Broadcasting 예시
print("=== Broadcasting 규칙 ===")

# 예1: 스칼라 vs 벡터
a = torch.tensor([1, 2, 3])
b = 10
print(f"a shape: {a.shape}, b shape: {b}")
print(f"a + b = {a + b}")
print(f"→ b가 (1,)에서 (3,)으로 자동 확장됨")

print("\n" + "="*50)

# 예2: (3, 1) vs (1, 5)
c = torch.randn(3, 1)
d = torch.randn(1, 5)
print(f"c shape: {c.shape}, d shape: {d.shape}")
result = c + d
print(f"c + d shape: {result.shape}")
print(f"→ (3, 1) + (1, 5) = (3, 5)")

print("\n" + "="*50)

# 예3: (batch, seq, emb) vs (emb,)
token_embeddings = torch.randn(2, 10, 768)  # (배치, 토큰, 임베딩)
bias = torch.randn(768)
print(f"token_embeddings shape: {token_embeddings.shape}")
print(f"bias shape: {bias.shape}")
result = token_embeddings + bias
print(f"결과 shape: {result.shape}")
print(f"→ 모든 배치, 모든 토큰에 같은 bias 적용됨")

## 7️⃣ Reshape vs View vs Permute 비교

텐서 형태를 변경하는 다양한 방법들

In [ ]:
# Reshape vs View
print("=== Reshape vs View ===")
x = torch.arange(12)  # [0, 1, 2, ..., 11]
print(f"원본: {x}, shape: {x.shape}")

# reshape: 메모리 레이아웃 무시하고 재구성
reshaped = x.reshape(3, 4)
print(f"\nreshape(3, 4):\n{reshaped}")

# view: 메모리 연속성 필요 (같은 메모리, 다른 해석)
viewed = x.view(3, 4)
print(f"\nview(3, 4):\n{viewed}")

print("\n일반적으로 reshape()을 사용하는 것이 안전합니다.")

In [ ]:
# 세 가지 방법의 차이
print("\n=== 세 가지 변형 메서드 비교 ===")
original = torch.randn(2, 3, 4)
print(f"원본 shape: {original.shape}")

# 1. reshape: 자유로운 재구성
reshaped = original.reshape(2, 12)  # (2, 3, 4) → (2, 12)
print(f"reshape(2, 12): {reshaped.shape}")

# 2. view: 메모리 연속성 필요 (같은 데이터 해석)
viewed = original.view(2, 12)
print(f"view(2, 12): {viewed.shape}")

# 3. permute: 차원 순서 변경 (데이터 이동)
permuted = original.permute(1, 0, 2)  # (2, 3, 4) → (3, 2, 4)
print(f"permute(1, 0, 2): {permuted.shape}")

print("\n핵심: reshape/view는 숫자 배열, permute는 차원 순서 변경")

## 8️⃣ 종합 실전 예시: GPT 어텐션 메커니즘

지금까지 배운 모든 개념을 적용한 실제 코드

In [ ]:
import torch
import torch.nn.functional as F

print("=== 간단한 멀티헤드 어텐션 구현 ===")

# 설정
batch_size = 2
seq_length = 4
emb_dim = 8
n_heads = 2
head_dim = emb_dim // n_heads

print(f"배치 크기: {batch_size}")
print(f"시퀀스 길이: {seq_length}")
print(f"임베딩 차원: {emb_dim}")
print(f"헤드 수: {n_heads}")
print(f"헤드 차원: {head_dim}\n")

# 입력 데이터
X = torch.randn(batch_size, seq_length, emb_dim)
print(f"입력 X shape: {X.shape}")

# 선형 변환 (Q, K, V 계산)
W_q = torch.randn(emb_dim, emb_dim)
W_k = torch.randn(emb_dim, emb_dim)
W_v = torch.randn(emb_dim, emb_dim)

# 1단계: 선형 변환
Q = X @ W_q  # (2, 4, 8)
K = X @ W_k  # (2, 4, 8)
V = X @ W_v  # (2, 4, 8)
print(f"\n1단계: 선형 변환")
print(f"Q shape: {Q.shape}, K shape: {K.shape}, V shape: {V.shape}")

# 2단계: 헤드로 분할
Q = Q.reshape(batch_size, seq_length, n_heads, head_dim)  # (2, 4, 2, 4)
K = K.reshape(batch_size, seq_length, n_heads, head_dim)
V = V.reshape(batch_size, seq_length, n_heads, head_dim)
print(f"\n2단계: 헤드로 분할")
print(f"Q shape: {Q.shape}")

# 3단계: 헤드별로 계산하기 위해 순서 변경
Q = Q.permute(0, 2, 1, 3)  # (2, 2, 4, 4) - [배치, 헤드, 시퀀스, 헤드_차원]
K = K.permute(0, 2, 1, 3)
V = V.permute(0, 2, 1, 3)
print(f"\n3단계: 차원 재배열")
print(f"Q shape: {Q.shape}")
print(f"이제 각 헤드별로 병렬 계산 가능!")

# 4단계: 어텐션 스코어 계산
# scores = Q @ K^T / sqrt(head_dim)
K_T = K.transpose(-2, -1)  # (2, 2, 4, 4) 마지막 두 차원 전치
scores = Q @ K_T / (head_dim ** 0.5)  # (2, 2, 4, 4)
print(f"\n4단계: 어텐션 스코어")
print(f"K.transpose(-2, -1) shape: {K_T.shape}")
print(f"scores = Q @ K^T / sqrt(head_dim)")
print(f"scores shape: {scores.shape}")

# 5단계: Softmax
weights = F.softmax(scores, dim=-1)  # (2, 2, 4, 4)
print(f"\n5단계: Softmax 가중치")
print(f"weights shape: {weights.shape}")

# 6단계: 값에 가중치 적용
output = weights @ V  # (2, 2, 4, 4) @ (2, 2, 4, 4) = (2, 2, 4, 4)
print(f"\n6단계: 값 가중치 적용")
print(f"output = weights @ V")
print(f"output shape: {output.shape}")

# 7단계: 헤드 다시 합치기
output = output.permute(0, 2, 1, 3)  # (2, 4, 2, 4)
print(f"\n7단계: 차원 재배열")
print(f"output.permute(0, 2, 1, 3) shape: {output.shape}")

# 8단계: 헤드 연결
output = output.reshape(batch_size, seq_length, emb_dim)  # (2, 4, 8)
print(f"\n8단계: 헤드 연결")
print(f"output.reshape(...) shape: {output.shape}")
print(f"\n최종 결과 shape: {output.shape} (입력과 동일!)")

## 9️⃣ 연산 복잡도 비교

다양한 연산의 효율성 비교

In [ ]:
import time

print("=== 연산 속도 비교 ===")

# 큰 텐서 생성
big_tensor = torch.randn(1000, 1000, 100)
print(f"테스트 텐서 shape: {big_tensor.shape}")

# 1. reshape
start = time.time()
for _ in range(100):
    _ = big_tensor.reshape(1000, 100000)
reshape_time = (time.time() - start) * 1000
print(f"reshape (100회): {reshape_time:.2f}ms")

# 2. view
linear_tensor = big_tensor.clone()
if linear_tensor.is_contiguous():
    start = time.time()
    for _ in range(100):
        _ = linear_tensor.view(1000, 100000)
    view_time = (time.time() - start) * 1000
    print(f"view (100회): {view_time:.2f}ms")

# 3. permute
start = time.time()
for _ in range(100):
    _ = big_tensor.permute(2, 0, 1)
permute_time = (time.time() - start) * 1000
print(f"permute (100회): {permute_time:.2f}ms")

print("\n💡 Insight: reshape와 view는 메타데이터만 변경해서 매우 빠르지만,")
print("   permute는 실제 데이터를 이동시켜 시간이 걸립니다.")

## 🔟 체크리스트: 텐서 연산 마스터하기

다음을 확인하세요:

In [ ]:
print("=== 개념 확인 체크리스트 ===")
print()

checklist = [
    ("✓", "Shape 읽기: (2, 4, 8)이 무엇을 의미하는지 설명 가능"),
    ("✓", "Indexing: x[0, :, :] vs x[:, 0, :] 차이점 이해"),
    ("✓", "Squeeze/Unsqueeze: 차원 제거/추가 목적 이해"),
    ("✓", "Transpose: 2D 행렬 전치와 permute로 고차원 전치 가능"),
    ("✓", "Dot Product: 벡터, 행렬, 배치 연산 모두 가능"),
    ("✓", "Broadcasting: 크기가 다른 텐서 연산 규칙 이해"),
    ("✓", "Reshape vs View: 언제 어떤 것을 사용할지 판단"),
    ("✓", "멀티헤드 어텐션: 8단계 프로세스 이해"),
]

for status, item in checklist:
    print(f"{status} {item}")

print("\n모두 이해했다면 LLM 코드도 쉬워집니다!")

## 참고: ch03, ch04의 코드에서 이런 개념들이 실제로 사용되는 곳

1. **ch03 - 어텐션 메커니즘**
   - `torch.matmul()` 및 `@`: 점곱으로 어텐션 스코어 계산
   - `transpose()`: Query와 Key 전치
   - Broadcasting: 스칼라 연산 (온도 조절 등)

2. **ch04 - GPT 모델**
   - `reshape()`: 멀티헤드 분할
   - `permute()`: 헤드 차원 재배열
   - `squeeze()`: 배치 1일 때 차원 제거
   - `unsqueeze()`: 마스크 차원 추가

3. **ch05 - 훈련**
   - 배치 처리 시 배치 차원 자동 확장
   - 손실 계산 시 배치/시퀀스 평균화